### Environment

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np
from decimal import Decimal

# System and timing utilities
import logging
import itertools as it

from pathlib import Path

# Custom utilities for Snowflake connection and Azure Data Lake Storage
from fsutils import run_sf_sql as rp

# Snowflake specific imports
from snowflake.connector.pandas_tools import write_pandas

from oxygen.conf.settings import settings
from oxygen.conf.context import context
from oxygen.exec.task import Task

import pandas as pd
import logging

import ipdb
import pdb

# Custom utilities for Snowflake connection and Azure Data Lake Storage
from fsutils import run_sf_sql as rp, config, adls_gen2

# from utils.utils import worker_output_table_validation
from multiprocessing import Pool, freeze_support

# Snowflake specific imports
from snowflake.connector.connection import SnowflakeConnection, SnowflakeCursor
from snowflake.connector.pandas_tools import write_pandas


# ================================
# DATABASE CONNECTION SETUP
# ================================

# Establish connection to Snowflake using notebook-xlarge warehouse for heavy processing
conn, _ = rp.get_connection("notebook-xlarge")

log = logging.getLogger(__name__)


In [ ]:
import pandas as pd
import os

os.chdir("..")
print(f"Current working directory: {os.getcwd()}")

### Bootstrap

In [ ]:
import os

from oxygen.boot import bootstrap
from oxygen.commands import *  # noqa

os.environ.setdefault("SETTINGS_MODULE", "configs.settings")
bootstrap()

### Input parameters

In [ ]:
from box.box_list import BoxList
import datetime

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S%f")

In [ ]:
CAT_NAME = 'BEVERAGES'
date = timestamp
need_state_table_wide = 'NEED_STATE_LATEST'
context.meta.run_id = '0b4612bf-3f96-46e8-a3a3-1cb2007ca35a'
full_scope_planogram = 'CORE_FS.PREPARED_MODEL.LOCALIZATION_POG_MASTER_FULL_SCOPE'
### Override the categories
context.data_stores.raw_data.categories = BoxList([CAT_NAME])


### configs need to override
# context.data_stores.raw_data.table_name.planogram_table = 'DL_FSCA_SLFSRV.TWA07.C890868_LOCALIZATION_POG_MASTER_0829_SALES_52W_202535'
# context.data_stores.raw_data.table_name.planogram_store_version = 'DL_FSCA_SLFSRV.TWA07.STORE_SPECIAL_COMBO_VERSION_VBACKUP'


# context.data_stores.raw_data.table_name.need_state_table = 'DL_FSCA_SLFSRV.TWA07.NEED_STATE_OPTI_HAIR_COLOR_0923'
# context.data_stores.raw_data.table_name.new_skus_table = 'DL_FSCA_SLFSRV.TWA07.FS_MANUAL_NEW_SKUS_1'

# context.data_stores.raw_data.table_name.elasticity_curve_table = 'DL_FSCA_SLFSRV.TWA07.SKU_HAIR_COLOR_20250923_WTIMESTAMP'

### Override the output table
# context.scaling_sales.output_tables.final_opti_output_scaled = f'SCALING_RESULT_V{timestamp}'

## Run in-scope scaling

In [ ]:
from core.tasks.optimization import (OptimizationScaling, OptimizationWaterfall)

In [ ]:
# OptimizationScaling().run()
# OptimizationWaterfall().run()

### Validate the scaling results

#### sales per store

In [ ]:
#### Check if new items sales per door
check_q = f"""
with new_sku_union as (
SELECT UPC as item_no_nbr,
    replaced_sku_number,
    estimated_annual_sales,
    estimated_door_count,
    div0(estimated_annual_sales,estimated_door_count) as "Estimated sales per door"
FROM {context.data_stores.raw_data.table_name.new_skus_table}
WHERE category in ('{"', '".join(context.data_stores.raw_data.categories)}')

UNION
SELECT DISTINCT product_id as item_no_nbr,
    NULL as replaced_sku_number,
    estimated_annual_sales,
    estimated_door_count,
    div0(estimated_annual_sales,estimated_door_count) as "Estimated sales per door"
FROM {context.data_stores.raw_data.table_name.need_state_table}
WHERE category in ('{"', '".join(context.data_stores.raw_data.categories)}')
and timestamp = (SELECT max(timestamp) FROM {context.data_stores.raw_data.table_name.need_state_table} 
                WHERE category in ('{"', '".join(context.data_stores.raw_data.categories)}'))
and "New Item" = True
)
,
new_sku_template as (
SELECT *
FROM new_sku_union
QUALIFY ROW_NUMBER() OVER (PARTITION BY item_no_nbr ORDER BY replaced_sku_number) = 1
)
,
raw_opti as (
SELECT item_no_nbr,
    SUM(n_stores * n_current_facings_sku) curr_facings,
    SUM(CASE WHEN n_current_facings_sku > 0 THEN n_stores ELSE 0 END) curr_pods,
    SUM(n_stores * n_current_projected_sales) cur_sales,
    SUM(n_stores * n_current_adjusted_projected_sales) cur_adj_sales,
    SUM(n_stores * n_optimal_facings_sku) opti_facings,
    SUM(CASE WHEN n_optimal_facings_sku > 0 THEN n_stores ELSE 0 END) opti_pods,
    SUM(n_stores * n_optimal_projected_sales) opti_sales,
    div0(SUM(n_stores * n_optimal_projected_sales),SUM(CASE WHEN n_optimal_facings_sku > 0 THEN n_stores ELSE 0 END)) as unadjusted_sales_per_door,
    SUM(n_stores * n_optimal_adjusted_projected_sales) opti_adj_sales,
    div0(SUM(n_stores * n_optimal_adjusted_projected_sales),SUM(CASE WHEN n_optimal_facings_sku > 0 THEN n_stores ELSE 0 END)) adjusted_sales_per_door
FROM DL_FSCA_SLFSRV.TWA07.{context.data_stores.output_data.table_name.final_opti_output}
WHERE run_id = '{context.meta.run_id}'
GROUP BY ALL
)
,
scaled_opti as (
SELECT item_no_nbr as item_no_nbr,
        SUM(n_current_scaled_adjusted_projected_sales * n_stores) cur_scaled_sales,
        SUM(n_optimal_scaled_adjusted_projected_sales * n_stores) opti_scaled_sales,
FROM DL_FSCA_SLFSRV.TWA07.{context.scaling_sales.output_tables.final_opti_output_scaled}
GROUP BY ALL
)
SELECT a.item_no_nbr, replaced_sku_number, estimated_annual_sales, estimated_door_count,"Estimated sales per door",
d.curr_facings as clone_item_curr_facings,
e.cur_scaled_sales as clone_item_curr_sales,
div0(e.cur_scaled_sales, d.curr_facings) as clone_item_sales_per_facing,
div0(e.cur_scaled_sales, d.curr_pods) as clone_item_sales_per_door,
b.curr_facings, b.curr_pods,b.cur_sales, b.cur_adj_sales,b.opti_facings,b.opti_pods, 
b.opti_sales, b.unadjusted_sales_per_door, b.opti_adj_sales, b.adjusted_sales_per_door , 
c.cur_scaled_sales, c.opti_scaled_sales
,div0(c.opti_scaled_sales,b.opti_pods) as opti_scaled_sales_per_door
,div0(c.cur_scaled_sales,b.curr_pods) as curr_scaled_sales_per_door
,div0(c.opti_scaled_sales,b.opti_facings) as opti_scaled_sales_per_facing
,div0(c.cur_scaled_sales,b.curr_facings) as curr_scaled_sales_per_facing
FROM new_sku_template a
LEFT JOIN raw_opti b ON a.item_no_nbr = b.item_no_nbr
LEFT JOIN scaled_opti c on a.item_no_nbr = c.item_no_nbr
LEFT JOIN raw_opti d ON a.replaced_sku_number = d.item_no_nbr
LEFT JOIN scaled_opti e on a.replaced_sku_number = e.item_no_nbr
;

"""

df_check = conn.cursor().execute(check_q).fetch_pandas_all()
df_check.columns = map(str.lower, df_check.columns)

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
## Clone SKU check
df_check.pipe(lambda dx: dx[~dx["replaced_sku_number"].isna()])[["item_no_nbr","clone_item_curr_facings","clone_item_curr_sales","clone_item_sales_per_door",
          "opti_pods","opti_facings",
          "opti_scaled_sales","opti_scaled_sales_per_door","opti_scaled_sales_per_facing","curr_scaled_sales_per_door"]]

In [ ]:
## Manual input
df_check.pipe(lambda dx: dx[dx["replaced_sku_number"].isna()])\
        .sort_values(by = "item_no_nbr")[["item_no_nbr","estimated sales per door","opti_pods","opti_facings","opti_scaled_sales_per_door","curr_scaled_sales_per_door"]]\
        .pipe(lambda dx: dx[~dx['estimated sales per door'].isna()])

#### DNP ratio

In [ ]:
check_q2 = f"""
with new_item_dnp as 
(
SELECT item_no_nbr, 
    vendor_name,
    median(dnp_ratio) as dnp_ratio_in_opti
FROM DL_FSCA_SLFSRV.TWA07.FNL_MICRO_OPTIMIZATION_RUNS_OB
WHERE run_id = '{context.meta.run_id}'
AND item_no_nbr in (
SELECT distinct UPC
FROM {context.data_stores.raw_data.table_name.new_skus_table}
WHERE category in ('{"', '".join(context.data_stores.raw_data.categories)}')
)
GROUP BY ALL
)
,
old_item_dnp as (
SELECT item_no_nbr,
    vendor_name,
    median(dnp_ratio) as replaced_sku_dnp_ratio
FROM DL_FSCA_SLFSRV.TWA07.FNL_MICRO_OPTIMIZATION_RUNS_OB
WHERE run_id = '{context.meta.run_id}'
AND item_no_nbr not in (
SELECT distinct UPC
FROM {context.data_stores.raw_data.table_name.new_skus_table}
WHERE category in ('{"', '".join(context.data_stores.raw_data.categories)}')
)
GROUP BY ALL
)
,
new_sku_template as (
SELECT * FROM {context.data_stores.raw_data.table_name.new_skus_table}
WHERE category in ('{"', '".join(context.data_stores.raw_data.categories)}')
)
SELECT a.*, b.sku_dnp_ratio, b.REPLACED_SKU_NUMBER, c.replaced_sku_dnp_ratio
FROM new_item_dnp a
LEFT JOIN new_sku_template b on a.item_no_nbr = b.upc
LEFT JOIN old_item_dnp c on b.REPLACED_SKU_NUMBER = c.item_no_nbr
"""

df_check2 = conn.cursor().execute(check_q2).fetch_pandas_all()
df_check2.columns = map(str.lower, df_check2.columns)

In [ ]:
df_check2

## Include out-of-scope data

### 4.1. items/plano sales info

In [ ]:
# ================================
# COMPREHENSIVE SALES TABLE CREATION
# ================================

# This query creates a comprehensive sales table that includes:
# - Items in scope per planogram
# - Stores in scope by category  
# - 2024 and L52 sales metrics (both in-scope and full-scope)
# - POD facing counts
# - DNP ratios
ACTUAL_SALES_TABLE_CATEGORY = 'ACTUAL_SALES_TABLE_CATEGORY'
items_sales_sql_query = f"""
CREATE OR REPLACE TABLE DL_FSCA_SLFSRV.TWA07.{ACTUAL_SALES_TABLE_CATEGORY} AS
WITH items_per_planogram AS (
    SELECT DISTINCT 
    plano_cat_desc as planogram_cat,
    fixture_desc AS planogram_name,
    item_no_nbr AS sku_nbr
    FROM DL_FSCA_SLFSRV.TWA07.FNL_MICRO_OPTIMIZATION_RUNS_OB
    WHERE TRUE
    AND RUN_ID = '{context.meta.run_id}'
)
,
-- CTE 2: Stores in scope by category
stores_in_scope AS (
    SELECT DISTINCT
        CAT_DSC AS planogram_cat,
        "store_nbr" AS store_nbr
    FROM {context.data_stores.raw_data.table_name.cluster_table}
), 
-- CTE 3: Valid planogram & store combinations
plano_in_store AS (
    SELECT DISTINCT
        PLANOGRAM_DSC as planogram_name,
        SKU_NBR,
        STORE_NBR
    FROM {context.data_stores.raw_data.table_name.planogram_table}
    WHERE PLANO_CAT_DSC = (CASE WHEN CAT_DSC = 'LAUNDRY' THEN 'HOUSEHOLD'
                                            WHEN CAT_DSC = 'SODA' THEN 'BEVERAGES'
                                            ELSE CAT_DSC
                        END)
        AND PLANOGRAM_NBR IN ('9605', '9200', '9600', '9584', '9350', '9330', 
                              '9617', '9500', '9378', '9569', '9232', '9711', 
                              '9380', '9576', '9136', '9040', '9061', '9030', 
                              '9041', '9140', '9281', '9260', '9280', '9210', 
                              '9640', '9542', '9544', '9700', '9677', '9751',
                              '9414', '9770', '9090', '9310', '9570', '9150', 
                              '9360', '9400', '9615', '9616', '9211', '9425')
),
-- CTE 3.1: All valid store-SKU rows in scope
in_scope AS (
    SELECT DISTINCT
        s.store_nbr,
        s.planogram_cat,
        i.planogram_name,
        i.sku_nbr
    FROM stores_in_scope s
    INNER JOIN items_per_planogram i ON i.planogram_cat = s.planogram_cat
    INNER JOIN plano_in_store p ON p.store_nbr = s.store_nbr 
        AND p.planogram_name = i.planogram_name 
        AND i.sku_nbr = p.sku_nbr
),
-- CTE 4: 2024 sales aggregated by store, SKU, and category
SKU_STORE_PANO_MASTER_DF AS (
SELECT SKU_NBR, STORE_NBR, max_by(planogram_dsc,
                                CASE WHEN planogram_nbr in (
                                '9605', '9200', '9600', '9584', '9350', '9330', 
                              '9617', '9500', '9378', '9569', '9232', '9711', 
                              '9380', '9576', '9136', '9040', '9061', '9030', 
                              '9041', '9140', '9281', '9260', '9280', '9210', 
                              '9640', '9542', '9544', '9700', '9677', '9751',
                              '9414', '9770', '9090', '9310', '9570', '9150', 
                              '9360', '9400', '9615', '9616', '9211', '9425') THEN 10000 ELSE planogram_nbr END
                                ) as planogram_dsc
FROM {full_scope_planogram}
WHERE TRUE 
AND CASE WHEN CAT_DSC = 'LAUNDRY' THEN 'HOUSEHOLD'
    WHEN CAT_DSC = 'SODA' THEN 'BEVERAGES'
    ELSE CAT_DSC
END in ('{"', '".join(context.data_stores.raw_data.categories)}')
AND planogram_nbr >= 9000
GROUP BY ALL
)
,
sales_2024 AS (
    SELECT
        STORE_NBR AS store_nbr,
        SKU_NBR AS sku_nbr,
        CAT_DSC,
        SUM(TOTAL_SALES) AS total_sales_2024,
        SUM(TOTAL_UNITS_SOLD) AS total_units_sold_2024
    FROM DL_FSCA_SLFSRV.TWA07.C835645_SALES_2024
    GROUP BY STORE_NBR, SKU_NBR, CAT_DSC
), 
-- CTE 5: 2024 sales for full scope (all categories)
SALES_2024_FULL_SCOPE AS (
    SELECT 
        s.SKU_NBR, 
        s.CAT_DSC,
        coalesce(p.planogram_dsc,'UNKNOWN') as PLANOGRAM_DSC,
        SUM(s.TOTAL_SALES) AS TOTAL_SALES_FULL_SCOPE_2024,
        SUM(s.TOTAL_UNITS_SOLD) as TOTAL_UNITS_SOLD_FULL_SCOPE_2024
    FROM DL_FSCA_SLFSRV.TWA07.C835645_SALES_2024 s
    LEFT JOIN SKU_STORE_PANO_MASTER_DF p on s.sku_nbr = p.sku_nbr and s.store_nbr = p.store_nbr
    WHERE TRUE
    GROUP BY ALL
    HAVING TOTAL_SALES_FULL_SCOPE_2024 > 0 OR TOTAL_UNITS_SOLD_FULL_SCOPE_2024 > 0
)
, 
-- CTE 6: L52 sales by store, SKU, and category
L52_SALES AS (
    SELECT 
        SKU_NBR, STORE_NBR, CAT_DSC,
        SUM(TOT_SCAN_SALES) AS TOTAL_SALES_L52,
        SUM(units_sold) as TOTAL_UNITS_SOLD_L52
    FROM APP_MERCH.PROD_PROMO.WKLY_SKU_STR_FEATURES a
    LEFT JOIN CORE_FSSC.CURATED_CALENDAR.FISCAL_WEEK b on a.fiscal_week_nbr = b.fiscal_week_nbr
    WHERE
        TRUE
        AND coalesce(ad_rgn_cd, 'NA') NOT IN ('F', 'W', '999')
        and a.fiscal_week_nbr between {context.scaling_sales.time_period.week_nbr_start} and {context.scaling_sales.time_period.week_nbr_end}
    GROUP BY SKU_NBR, STORE_NBR, CAT_DSC
), 
-- CTE 7: L52 sales for full scope
L52_FULL_SCOPE AS (
    SELECT s.SKU_NBR, 
            s.CAT_DSC,
            coalesce(p.planogram_dsc,'UNKNOWN') as PLANOGRAM_DSC,
        SUM(s.TOTAL_SALES_L52) AS TOTAL_SALES_FULL_SCOPE_L52,
        SUM(s.TOTAL_UNITS_SOLD_L52) as TOTAL_UNITS_SOLD_FULL_SCOPE_L52
    FROM L52_SALES s
    LEFT JOIN SKU_STORE_PANO_MASTER_DF p on s.sku_nbr = p.sku_nbr and s.store_nbr = p.store_nbr
    WHERE TRUE
    GROUP BY ALL
    HAVING TOTAL_SALES_FULL_SCOPE_L52 > 0 OR TOTAL_UNITS_SOLD_FULL_SCOPE_L52 > 0
), 
-- CTE 9: Store & SKU view combining all metrics
store_sku_view as (
    SELECT
        isc.sku_nbr,
        isc.store_nbr,
        isc.planogram_name,
        isc.planogram_cat,
        COALESCE(s.total_sales_2024, 0) AS total_sales_2024,
        COALESCE(s.total_units_sold_2024, 0) AS total_units_sold_2024,
        COALESCE(lfts.TOTAL_SALES_L52, 0) AS total_sales_l52,
        COALESCE(lfts.TOTAL_UNITS_SOLD_L52, 0) AS total_units_sold_l52
    FROM in_scope isc
    LEFT JOIN sales_2024 s ON s.store_nbr=isc.store_nbr AND s.sku_nbr=isc.sku_nbr
    LEFT JOIN L52_SALES lfts ON lfts.store_nbr=isc.store_nbr AND lfts.sku_nbr=isc.sku_nbr
), 
-- CTE 10: In-scope metrics aggregated
IN_SCOPE_FINAL AS (
    SELECT
        SKU_NBR,
        PLANOGRAM_name AS PLANOGRAM_DSC, 
        planogram_cat,
        SUM(total_sales_2024) AS ACTUAL_SALES_IN_SCOPE_2024,
        SUM(total_units_sold_2024) AS ACTUAL_UNITS_SOLD_IN_SCOPE_2024,
        SUM(total_sales_l52) AS ACTUAL_SALES_IN_SCOPE_L52,
        SUM(total_units_sold_l52) AS ACTUAL_UNITS_SOLD_IN_SCOPE_L52
    FROM store_sku_view
    GROUP BY SKU_NBR, PLANOGRAM_name, planogram_cat
)
, 
-- CTE 11: Full scope metrics combined
FULL_SCOPE_FINAL AS (
    SELECT a.SKU_NBR, a.CAT_DSC,a.PLANOGRAM_DSC,
           COALESCE(TOTAL_SALES_FULL_SCOPE_L52, 0) AS TOTAL_SALES_FULL_SCOPE_L52,
           COALESCE(TOTAL_UNITS_SOLD_FULL_SCOPE_L52, 0) AS TOTAL_UNITS_SOLD_FULL_SCOPE_L52,
           COALESCE(TOTAL_SALES_FULL_SCOPE_2024, 0) AS TOTAL_SALES_FULL_SCOPE_2024,
           COALESCE(TOTAL_UNITS_SOLD_FULL_SCOPE_2024, 0) AS TOTAL_UNITS_SOLD_FULL_SCOPE_2024
    FROM L52_FULL_SCOPE a
    FULL OUTER JOIN SALES_2024_FULL_SCOPE b USING (SKU_NBR, PLANOGRAM_DSC)
    WHERE TRUE
    AND CASE WHEN CAT_DSC = 'LAUNDRY' THEN 'HOUSEHOLD'
    WHEN CAT_DSC = 'SODA' THEN 'BEVERAGES'
    ELSE CAT_DSC
END in ('{"', '".join(context.data_stores.raw_data.categories)}')
)
, 
-- CTE 12: Last 12 months DNP data
L12M_DNP AS (
    SELECT TOP 12 FISCAL_MONTH_NBR
    FROM (SELECT DISTINCT FISCAL_MONTH_NBR FROM PL_MBU.EXT_APP.V_CR_SKU_SLS_SPACE_DNP_MNTH_FNL)
    ORDER BY FISCAL_MONTH_NBR DESC
), 
-- CTE 13: Calculate DNP ratios
BASE_DNP AS (
    SELECT SKU_NBR,
           DIV0(SUM(TOT_DNP_AMT), SUM(TOTAL_SALES)) AS ITEM_LATEST_DNP_RATIO
    FROM PL_MBU.EXT_APP.V_CR_SKU_SLS_SPACE_DNP_MNTH_FNL
    WHERE FISCAL_MONTH_NBR IN (SELECT FISCAL_MONTH_NBR FROM L12M_DNP)
    GROUP BY SKU_NBR
)

-- Final SELECT: Combine all metrics
SELECT FSC.*,
       -- COALESCE(ISC.ACTUAL_QTY_STORES_IN_SCOPE, 0) AS ACTUAL_QTY_STORES_IN_SCOPE,
       -- COALESCE(ISC.ACTUAL_PODS_IN_SCOPE, 0) AS ACTUAL_PODS_IN_SCOPE,
       COALESCE(ISC.ACTUAL_SALES_IN_SCOPE_2024, 0) AS ACTUAL_SALES_IN_SCOPE_2024,
       COALESCE(ISC.ACTUAL_UNITS_SOLD_IN_SCOPE_2024, 0) AS ACTUAL_UNITS_SOLD_IN_SCOPE_2024,
       COALESCE(ISC.ACTUAL_SALES_IN_SCOPE_L52, 0) AS ACTUAL_SALES_IN_SCOPE_L52,
       COALESCE(ISC.ACTUAL_UNITS_SOLD_IN_SCOPE_L52, 0) AS ACTUAL_UNITS_SOLD_IN_SCOPE_L52,
       B.ITEM_LATEST_DNP_RATIO
FROM FULL_SCOPE_FINAL FSC
LEFT JOIN IN_SCOPE_FINAL ISC USING (SKU_NBR, PLANOGRAM_DSC)
LEFT JOIN BASE_DNP B USING(SKU_NBR)
;
"""
# Execute the comprehensive sales table creation
print(f"Creating comprehensive sales table...to {ACTUAL_SALES_TABLE_CATEGORY}")
conn.cursor().execute(items_sales_sql_query)

### 4.2 Linear space

In [ ]:
# ================================
# CALCULATING FACINGS AND LINEAR SPACE
# ================================

print("Calculating facings and linear space for:")
print("  - IN SCOPE: Items and stores included in optimization")
print("  - OUT OF SCOPE: Items and stores excluded from optimization") 
print("  - TOTAL: Combined metrics across all items and stores")
print("")
ACTUAL_LINEAR_SPACE_TABLE = 'ACTUAL_LINEAR_SPACE_TABLE'
facings_linear_space_sql_query = f"""
CREATE OR REPLACE TABLE DL_FSCA_SLFSRV.TWA07.{ACTUAL_LINEAR_SPACE_TABLE} AS
WITH 
items_per_planogram AS (
    SELECT DISTINCT 
    plano_cat_desc as planogram_cat,
    fixture_desc AS planogram_name,
    item_no_nbr AS sku_nbr
    FROM DL_FSCA_SLFSRV.TWA07.FNL_MICRO_OPTIMIZATION_RUNS_OB
    WHERE TRUE
    AND RUN_ID = '{context.meta.run_id}'
)
,

-- Stores that are “in scope” for each category
stores_in_scope AS (
    SELECT DISTINCT
        CAT_DSC AS planogram_cat,
        "store_nbr" as store_nbr
    FROM {context.data_stores.raw_data.table_name.cluster_table}
),

-- Valid (store‑planogram‑SKU) rows that physically exist on‑shelf
plano_in_store AS (
    SELECT DISTINCT
        STORE_NBR AS store_nbr,
        PLANOGRAM_DSC AS planogram_name,
        SKU_NBR AS sku_nbr
    FROM {context.data_stores.raw_data.table_name.planogram_table}
    WHERE PLANO_CAT_DSC = (CASE WHEN CAT_DSC = 'LAUNDRY' THEN 'HOUSEHOLD'
                                            WHEN CAT_DSC = 'SODA' THEN 'BEVERAGES'
                                            ELSE CAT_DSC
                        END)
      AND PLANOGRAM_NBR IN (
           '9605','9200','9600','9584','9350','9330','9617','9500','9378','9569','9232','9711',
           '9380','9576','9136','9040','9061','9030','9041','9140','9281','9260','9280','9210',
           '9640','9542','9544','9700','9677','9751','9414','9770','9090','9310','9570','9150',
           '9360','9400','9615','9616','9211','9425'
      )
),

-- Fully resolved “in scope” rows
in_scope AS (
    SELECT DISTINCT
        s.store_nbr,
        i.planogram_name,
        i.sku_nbr
    FROM stores_in_scope s
    JOIN items_per_planogram i ON i.planogram_cat = s.planogram_cat
    JOIN plano_in_store p ON p.store_nbr = s.store_nbr
                                  AND p.planogram_name = i.planogram_name
                                  AND p.sku_nbr = i.sku_nbr
),

-- Raw facings pulled from the master POG file
pog_facings AS (
    SELECT
        STORE_NBR AS store_nbr,
        PLANOGRAM_DSC AS planogram_name,
        SKU_NBR AS sku_nbr,
        SUM(HORIZONTAL_FACINGS_NBR) AS facings
    FROM {full_scope_planogram}
    WHERE planogram_nbr >= 9000
    AND CASE WHEN CAT_DSC = 'LAUNDRY' THEN 'HOUSEHOLD'
                                            WHEN CAT_DSC = 'SODA' THEN 'BEVERAGES'
                                            ELSE CAT_DSC
    END in ('{"', '".join(context.data_stores.raw_data.categories)}')
    GROUP BY STORE_NBR, PLANOGRAM_DSC, SKU_NBR
)
,
-- Facings flagged by scope
pog_facings_scoped AS (
    SELECT
        p.*,
        CASE WHEN i.sku_nbr IS NOT NULL 
                and i.planogram_name IS NOT NULL 
                and i.sku_nbr IS NOT NULL THEN 1 ELSE 0 END AS in_scope_flag
    FROM pog_facings p
    LEFT JOIN in_scope i
           ON i.store_nbr = p.store_nbr
          AND i.planogram_name = p.planogram_name
          AND i.sku_nbr = p.sku_nbr
)
,

-- FINAL CTE — one row per SKU with requested metrics
sku_facings_space AS (
    SELECT
        p.sku_nbr AS item_no_nbr,
        p.planogram_name,
        -- Facings
        SUM(p.facings) AS facing_total,
        SUM(CASE WHEN in_scope_flag = 1 THEN p.facings ELSE 0 END) AS facings_in_scope,
        SUM(CASE WHEN in_scope_flag = 0 THEN p.facings ELSE 0 END) AS facings_out_of_scope,
        
        -- Linear space = facings × unit width
        SUM(p.facings * COALESCE(s.unit_wth/12,0)) AS linear_space_total,
        SUM(CASE WHEN in_scope_flag = 1 THEN p.facings * COALESCE(s.unit_wth/12,0) ELSE 0 END) AS linear_space_in_scope,
        SUM(CASE WHEN in_scope_flag = 0 THEN p.facings * COALESCE(s.unit_wth/12,0) ELSE 0 END) AS linear_space_out_of_scope,

        -- POGs, stores with facings
        COUNT(DISTINCT CASE WHEN p.facings > 0 THEN STORE_NBR ELSE NULL END) AS pods_total,
        COUNT(DISTINCT CASE WHEN p.facings > 0 AND in_scope_flag = 1 THEN STORE_NBR ELSE NULL END) AS pod_in_scope,
        COUNT(DISTINCT CASE WHEN p.facings > 0 AND in_scope_flag = 0 THEN STORE_NBR ELSE NULL END) AS pod_out_of_scope
        
    FROM pog_facings_scoped p
    LEFT JOIN CORE_FS.CURATED_PRODUCT.SKU s ON s.sku_nbr = p.sku_nbr
    GROUP BY p.sku_nbr,p.planogram_name
)

SELECT *
FROM sku_facings_space
"""
# Execute the comprehensive sales table creation
print(f"Creating linear space table...to {ACTUAL_LINEAR_SPACE_TABLE}")
conn.cursor().execute(facings_linear_space_sql_query)


## 5. Concat the results

### 5.1 SKU level

In [ ]:
SKU_level_query = f"""
WITH OPT_DF as (
SELECT *
FROM DL_FSCA_SLFSRV.TWA07.{context.scaling_sales.output_tables.final_opti_output_scaled}
WHERE TRUE
AND RUN_ID = '{context.meta.run_id}'
)
,
ITEM_DESCRIPTION AS (
    SELECT DISTINCT 
        SKU_NBR AS       item_no_nbr,
        CVS_ITEM_DSC as  item_no_dsc,
        CAT_DSC AS CATEGORY,
        seg_dsc as segment,
        VENDOR_NBR,
        VENDOR_NAME,
        CM_BRAND_DSC AS BRAND_NAME,
        SUBCAT_DSC AS SUB_CATEGORY,
        UNIT_WTH/12 AS ITEM_WIDTH_FEET
    FROM CORE_FS.CURATED_PRODUCT.SKU
), 
-- CTE 1: Get the full scope
FULL_SCOPE as (
SELECT DISTINCT SKU_NBR, PLANOGRAM_DSC
FROM DL_FSCA_SLFSRV.TWA07.{ACTUAL_SALES_TABLE_CATEGORY}
UNION
SELECT DISTINCT item_no_nbr AS SKU_NBR, fixture_desc AS PLANOGRAM_DSC
FROM OPT_DF
)
,
-- CTE 2: Aggregate metrics at item level
ITEM_AGGREGATE_METRICS AS (
    SELECT 
        'baseline' as   run_id, 
        run_id as       raw_run_id, 
        fixture_desc AS PLANOGRAM_DSC, 
        item_no_nbr as  item_no_nbr,
        
        -- POD (Plan-O-Gram Distribution) calculations
        -- Count stores where item has facings in each state
        SUM(CASE WHEN n_optimal_facings_sku > 0 THEN n_stores ELSE 0 END) AS OPTIMAL_POD,
        SUM(n_optimal_facings_sku * n_stores) AS OPTIMAL_FACING_COUNT,

        SUM(CASE WHEN n_current_facings_sku > 0 THEN n_stores ELSE 0 END) AS CURRENT_POD,
        SUM(n_current_facings_sku * n_stores) AS CURRENT_FACING_COUNT,

        SUM(CASE WHEN n_upcoming_facings_sku > 0 THEN n_stores ELSE 0 END) AS UPCOMING_POD,
        SUM(n_upcoming_facings_sku * n_stores) AS UPCOMING_FACING_COUNT,
        
        -- Sales projections (normalized by store count)
        SUM(n_optimal_scaled_adjusted_projected_sales  * n_stores) AS normalized_optimal_adjusted_projected_sales,
        SUM(n_current_scaled_adjusted_projected_sales  * n_stores) AS normalized_current_adjusted_projected_sales,
        SUM(n_upcoming_scaled_adjusted_projected_sales * n_stores) AS normalized_upcoming_adjusted_projected_sales,
        
        -- DNP projections
        SUM(n_optimal_scaled_adjusted_projected_dnp  * n_stores) AS normalized_optimal_adjusted_projected_dnp,
        SUM(n_current_scaled_adjusted_projected_dnp  * n_stores) AS normalized_current_adjusted_projected_dnp,
        SUM(n_upcoming_scaled_adjusted_projected_dnp * n_stores) AS normalized_upcoming_adjusted_projected_dnp,

        -- Unit sold
        SUM(n_optimal_scaled_adjusted_projected_units_sold  * n_stores) as optimal_units_sold,
        SUM(n_current_scaled_adjusted_projected_units_sold  * n_stores) as current_units_sold,
        SUM(n_upcoming_scaled_adjusted_projected_units_sold * n_stores) as upcoming_units_sold,
        
        SUM(n_optimal_linear_space_used_sku * n_stores) as optimal_linear_space,
        sum(n_current_linear_space_used_sku * n_stores) as n_current_linear_space_used_sku,
        min({context.scaling_sales.time_period.week_nbr_end})  as adjustment_max_fiscal_week,
        
        MIN(dnp_ratio) AS dnp_ratio  -- Take minimum DNP ratio for the item
    FROM OPT_DF
    GROUP BY ALL
)

, 
GRANULARITY as (
SELECT DISTINCT
        'baseline' as run_id, 
        run_id as raw_run_id,
        'baseline' AS SCENEARIO_OPTIMIZATION_VENDOR,
        '' AS SPACE_CHANGE,
        '-' AS PCT_SPACE_CHANGE
        FROM OPT_DF
)
,
-- CTE 3: Base item information from optimization runs
BASE_ITEM_TABLE AS (
    SELECT DISTINCT 
        'baseline' as run_id, 
        run_id as raw_run_id,
        'baseline' AS SCENEARIO_OPTIMIZATION_VENDOR,
        '' AS SPACE_CHANGE,
        '-' AS PCT_SPACE_CHANGE,
        fixture_desc AS PLANOGRAM_DSC,
        item_no_nbr AS item_no_nbr,
        item_no_desc AS ITEM_NO_DSC,
        seg_dsc AS SEGMENT,
        -- Extract choice map from need state ID
        REGEXP_SUBSTR(need_state_unique_id, '^(.*)_', 1, 1, 'e') AS CHOICE_MAP
    FROM OPT_DF
)
, 
-- CTE 4: Get unit prices for unit sales calculations
UNIT_PRICE_TBL AS (
    SELECT DISTINCT
        UPC AS ITEM_NO_NBR,
        SKU_MSRP AS UNIT_PRICE
    FROM {context.data_stores.raw_data.table_name.new_skus_table}
    WHERE UPC not in (SELECT SKU_NBR FROM DL_FSCA_SLFSRV.TWA07.STORE_SKU_UNIT_PRICE)
    
    UNION
    
    SELECT 
        SKU_NBR AS item_no_nbr,
        MEDIAN(L52W_PRICE) AS UNIT_PRICE
    FROM DL_FSCA_SLFSRV.TWA07.STORE_SKU_UNIT_PRICE
    GROUP BY SKU_NBR
)
, 
-- CTE 5: Get actual sales data from comprehensive sales table
RELEVANT_ACTUALS AS (
    SELECT 
        f.SKU_NBR AS item_no_nbr,
        sku.category_dsc AS CATEGORY,
        f.PLANOGRAM_DSC,
        coalesce(s.TOTAL_SALES_FULL_SCOPE_L52,0)       AS TOTAL_SALES_FULL_SCOPE_L52,
        coalesce(s.TOTAL_UNITS_SOLD_FULL_SCOPE_L52,0)  AS TOTAL_UNITS_SOLD_FULL_SCOPE_L52,
        coalesce(s.TOTAL_SALES_FULL_SCOPE_2024,0)      AS TOTAL_SALES_FULL_SCOPE_2024,
        coalesce(s.TOTAL_UNITS_SOLD_FULL_SCOPE_2024,0) AS TOTAL_UNITS_SOLD_FULL_SCOPE_2024,
        coalesce(s.ACTUAL_SALES_IN_SCOPE_2024,0)       AS ACTUAL_SALES_IN_SCOPE_2024,
        coalesce(s.ACTUAL_UNITS_SOLD_IN_SCOPE_2024,0)  AS ACTUAL_UNITS_SOLD_IN_SCOPE_2024,
        coalesce(s.ACTUAL_SALES_IN_SCOPE_L52,0)        AS ACTUAL_SALES_IN_SCOPE_L52,
        coalesce(s.ACTUAL_UNITS_SOLD_IN_SCOPE_L52,0)   AS ACTUAL_UNITS_SOLD_IN_SCOPE_L52,
        coalesce(s.ITEM_LATEST_DNP_RATIO, NULL)        AS ITEM_LATEST_DNP_RATIO
    FROM FULL_SCOPE f
    LEFT JOIN DL_FSCA_SLFSRV.TWA07.{ACTUAL_SALES_TABLE_CATEGORY} s 
    on f.SKU_NBR = s.sku_nbr and f.PLANOGRAM_DSC = s.PLANOGRAM_DSC
    LEFT JOIN CORE_FSSC.CURATED_PRODUCT.SKU sku on f.SKU_NBR = sku.sku_nbr
)
,

-- Linear space
LINEAR_SPACE_ACTUAL as (
SELECT  f.SKU_NBR AS item_no_nbr,
        f.PLANOGRAM_DSC,
        coalesce(FACING_TOTAL,0) AS FACING_TOTAL,
        coalesce(FACINGS_IN_SCOPE,0) AS FACINGS_IN_SCOPE,
        coalesce(FACINGS_OUT_OF_SCOPE,0) AS FACINGS_OUT_OF_SCOPE,
        coalesce(LINEAR_SPACE_TOTAL,0) AS LINEAR_SPACE_TOTAL,
        coalesce(LINEAR_SPACE_IN_SCOPE,0) AS LINEAR_SPACE_IN_SCOPE,
        coalesce(LINEAR_SPACE_OUT_OF_SCOPE,0) AS LINEAR_SPACE_OUT_OF_SCOPE,
        coalesce(PODS_TOTAL,0) AS PODS_TOTAL,
        coalesce(POD_IN_SCOPE,0) AS POD_IN_SCOPE,
        coalesce(POD_OUT_OF_SCOPE,0) AS POD_OUT_OF_SCOPE
FROM FULL_SCOPE f
LEFT JOIN DL_FSCA_SLFSRV.TWA07.{ACTUAL_LINEAR_SPACE_TABLE} s
on f.SKU_NBR = s.item_no_nbr and f.PLANOGRAM_DSC = s.PLANOGRAM_NAME
)
,
-- new items
new_item_df as
(
SELECT DISTINCT
    n.*,
    v.vendor_nbr, 
    pv.pay_vendor_nm
FROM {context.data_stores.raw_data.table_name.new_skus_table} n
LEFT JOIN CORE_FSSC.CURATED_PRODUCT.VENDOR v on n.vendor_number = v.vendor_nbr
LEFT JOIN CORE_FSSC.CURATED_PRODUCT.PAY_VENDOR pv on v.pay_vendor_nbr = pv.pay_vendor_nbr
)

,
item_pano_df as 
(SELECT  
        -- GRANULARITY
        G.run_id, 
        G.SCENEARIO_OPTIMIZATION_VENDOR as sceneario_optimization_vendor,
        G.SPACE_CHANGE as space_change,
        G.PCT_SPACE_CHANGE as pct_space_change,
        RA.PLANOGRAM_DSC as planogram_dsc,
        -- Actauls
        RA.item_no_nbr,
        coalesce(ID.item_no_dsc, ND.product_name) as  item_no_dsc,
        coalesce(RA.CATEGORY, ND.category) as category,
        coalesce(ID.SUB_CATEGORY, ND.subcategory) as subcategory,
        coalesce(ID.segment, ND.segment) as segment,
        B.CHOICE_MAP as choice_map,
        coalesce(pv.pay_vendor_nm, ND.pay_vendor_nm) as pay_vendor_name,
        coalesce(ID.BRAND_NAME, ND.brand_name) as brand_name,
        ID.ITEM_WIDTH_FEET as item_width_feet,
        
        -- ITEM CATEGORY, ATTRIBUTES
        IAM.OPTIMAL_FACING_COUNT  AS projected_optimal_facing_count,
        IAM.OPTIMAL_POD           AS projected_optimal_pod,
        IAM.CURRENT_FACING_COUNT  AS projected_current_facing_count,
        IAM.CURRENT_POD           AS projected_current_pod,
        IAM.UPCOMING_FACING_COUNT AS projected_upcoming_facing_count,
        IAM.UPCOMING_POD          AS projected_upcoming_pod,
         -- Sales projections
        IAM.normalized_optimal_adjusted_projected_sales as normalized_optimal_projected_sales,
        IAM.normalized_optimal_adjusted_projected_sales,
        IAM.normalized_current_adjusted_projected_sales as normalized_current_projected_sales,
        IAM.normalized_current_adjusted_projected_sales,
        IAM.normalized_upcoming_adjusted_projected_sales as normalized_upcoming_projected_sales,
        IAM.normalized_upcoming_adjusted_projected_sales,
         -- DNP projections
        IAM.normalized_optimal_adjusted_projected_dnp as normalized_optimal_projected_dnp,
        IAM.normalized_optimal_adjusted_projected_dnp,
        IAM.normalized_current_adjusted_projected_dnp as normalized_current_projected_dnp,
        IAM.normalized_current_adjusted_projected_dnp,
        IAM.normalized_upcoming_adjusted_projected_dnp as normalized_upcoming_projected_dnp,
        IAM.normalized_upcoming_adjusted_projected_dnp,
        

        -- DNP
        coalesce(RA.ITEM_LATEST_DNP_RATIO,IAM.dnp_ratio) as dnp_ratio,
        UPT.UNIT_PRICE as unit_price,

        -- Actuals - In scope
        
        LSA.POD_IN_SCOPE as actual_qty_stores_in_scope,
        LSA.POD_IN_SCOPE as actual_pods_in_scope,
        RA.ACTUAL_SALES_IN_SCOPE_2024 as actual_sales_in_scope_2024,
        RA.ACTUAL_SALES_IN_SCOPE_2024 * RA.ITEM_LATEST_DNP_RATIO as dnp_in_scope_2024,
        RA.ACTUAL_UNITS_SOLD_IN_SCOPE_2024 as actual_units_sold_in_scope_2024,

        RA.ACTUAL_SALES_IN_SCOPE_L52 as  actual_sales_in_scope_l52,
        RA.ACTUAL_UNITS_SOLD_IN_SCOPE_L52 as actual_units_sold_in_scope_l52,
        RA.ACTUAL_SALES_IN_SCOPE_L52 * RA.ITEM_LATEST_DNP_RATIO as dnp_in_scope_l52,

        IAM.optimal_units_sold,
        IAM.current_units_sold,
        IAM.upcoming_units_sold,
        IAM.adjustment_max_fiscal_week,
    
        LSA.FACING_TOTAL as facing_total,
        LSA.FACINGS_IN_SCOPE as facings_in_scope,
        LSA.FACINGS_OUT_OF_SCOPE as facings_out_of_scope,

        LSA.LINEAR_SPACE_TOTAL        as linear_space_total,
        LSA.LINEAR_SPACE_IN_SCOPE     as linear_space_in_scope,
        LSA.LINEAR_SPACE_OUT_OF_SCOPE as linear_space_out_of_scope,
        
        
        RA.TOTAL_SALES_FULL_SCOPE_L52 as sales_total,
        RA.TOTAL_SALES_FULL_SCOPE_L52 - RA.ACTUAL_SALES_IN_SCOPE_L52 as sales_out_of_scope,

        RA.TOTAL_UNITS_SOLD_FULL_SCOPE_L52 as units_sold_total,
        RA.TOTAL_UNITS_SOLD_FULL_SCOPE_L52 - RA.ACTUAL_UNITS_SOLD_IN_SCOPE_L52 as units_sold_out_of_scope,
        
        LSA.PODS_TOTAL as pods_total,
        LSA.POD_OUT_OF_SCOPE as pod_out_of_scope,

        RA.TOTAL_SALES_FULL_SCOPE_L52 * coalesce(RA.ITEM_LATEST_DNP_RATIO,IAM.dnp_ratio) as dnp_total,
        (RA.TOTAL_SALES_FULL_SCOPE_L52 - RA.ACTUAL_SALES_IN_SCOPE_L52) * coalesce(RA.ITEM_LATEST_DNP_RATIO,IAM.dnp_ratio) as dnp_out_of_scope,

        RA.TOTAL_SALES_FULL_SCOPE_2024 as actual_sales_total_2024,
        RA.TOTAL_SALES_FULL_SCOPE_2024 - RA.ACTUAL_SALES_IN_SCOPE_2024 as actual_sales_out_of_scope_2024,
        
        
        -- opt + out of scope
         coalesce(IAM.normalized_optimal_adjusted_projected_sales,0) + (RA.TOTAL_SALES_FULL_SCOPE_L52 - RA.ACTUAL_SALES_IN_SCOPE_L52) as adj_opti_plus_out_of_scope_sales,
         coalesce(IAM.normalized_optimal_adjusted_projected_dnp,0) +   (RA.TOTAL_SALES_FULL_SCOPE_L52 - RA.ACTUAL_SALES_IN_SCOPE_L52) * coalesce(RA.ITEM_LATEST_DNP_RATIO,IAM.dnp_ratio)  as adj_opti_plus_out_of_scope_dnp,
         coalesce(IAM.optimal_units_sold,0) + (RA.TOTAL_UNITS_SOLD_FULL_SCOPE_L52 - RA.ACTUAL_UNITS_SOLD_IN_SCOPE_L52) as adj_opti_plus_out_of_scope_units_sold,
         coalesce(IAM.OPTIMAL_POD,0) + LSA.POD_OUT_OF_SCOPE as adj_opti_plus_out_of_scope_pods,
         coalesce(IAM.OPTIMAL_FACING_COUNT,0) + LSA.FACINGS_OUT_OF_SCOPE as adj_opti_plus_out_of_scope_facings,
         coalesce(IAM.optimal_linear_space,0) + LSA.LINEAR_SPACE_OUT_OF_SCOPE  as adj_opti_plus_out_of_scope_linear_space,
         coalesce(IAM.optimal_linear_space,0) as optimal_linear_space,
         coalesce(IAM.n_current_linear_space_used_sku,0) as n_current_linear_space_used_sku,

        -- current + out of scope
         coalesce(IAM.CURRENT_FACING_COUNT,0) + coalesce(LSA.FACINGS_OUT_OF_SCOPE,0) as current_plus_out_of_scope_facings
FROM RELEVANT_ACTUALS RA
CROSS JOIN GRANULARITY G
LEFT JOIN ITEM_DESCRIPTION ID ON RA.item_no_nbr = ID.item_no_nbr
LEFT JOIN CORE_FSSC.CURATED_PRODUCT.VENDOR v on ID.vendor_nbr = v.vendor_nbr
LEFT JOIN CORE_FSSC.CURATED_PRODUCT.PAY_VENDOR pv on v.pay_vendor_nbr = pv.pay_vendor_nbr
LEFT JOIN new_item_df ND ON RA.item_no_nbr = ND.upc
LEFT JOIN UNIT_PRICE_TBL UPT on RA.item_no_nbr = UPT.item_no_nbr
LEFT JOIN LINEAR_SPACE_ACTUAL LSA 
ON RA.item_no_nbr = LSA.item_no_nbr and RA.PLANOGRAM_DSC = LSA.PLANOGRAM_DSC
LEFT JOIN ITEM_AGGREGATE_METRICS IAM
on RA.item_no_nbr = IAM.item_no_nbr and RA.PLANOGRAM_DSC = IAM.PLANOGRAM_DSC and G.run_id = IAM.run_id
LEFT JOIN BASE_ITEM_TABLE B
on RA.item_no_nbr = B.item_no_nbr and RA.PLANOGRAM_DSC = B.PLANOGRAM_DSC and G.run_id = B.run_id)

,
vendor_level_df as (
SELECT  
        run_id, 
        SPLIT_PART(run_id, '_', 2) AS SCENEARIO_OPTIMIZATION_VENDOR,
        SPLIT_PART(run_id, '_', 4) AS SPACE_CHANGE,
        CONCAT(SPLIT_PART(run_id, '_', 5), '-', SPLIT_PART(run_id, '_', 6)) AS PCT_SPACE_CHANGE,
            -- Sales projections
        sum(normalized_optimal_projected_sales) as normalized_optimal_projected_sales,
        sum(normalized_optimal_adjusted_projected_sales) as normalized_optimal_adjusted_projected_sales,
        sum(normalized_current_projected_sales) as normalized_current_projected_sales,
        sum(normalized_current_adjusted_projected_sales) as normalized_current_adjusted_projected_sales,
        sum(normalized_upcoming_projected_sales) as normalized_upcoming_projected_sales,
        sum(normalized_upcoming_adjusted_projected_sales) as normalized_upcoming_adjusted_projected_sales,
         -- DNP projections
        sum(normalized_optimal_projected_dnp) as normalized_optimal_projected_dnp,
        sum(normalized_optimal_adjusted_projected_dnp) as normalized_optimal_adjusted_projected_dnp,
        sum(normalized_current_projected_dnp) as normalized_current_projected_dnp,
        sum(normalized_current_adjusted_projected_dnp) as normalized_current_adjusted_projected_dnp,
        sum(normalized_upcoming_projected_dnp) as normalized_upcoming_projected_dnp,
        sum(normalized_upcoming_adjusted_projected_dnp) as normalized_upcoming_adjusted_projected_dnp,
        

        -- Actuals - In scope
        
        sum(actual_qty_stores_in_scope) as actual_qty_stores_in_scope,
        sum(actual_pods_in_scope) as actual_pods_in_scope,
        sum(actual_sales_in_scope_2024) as actual_sales_in_scope_2024,
        sum(dnp_in_scope_2024) as dnp_in_scope_2024,
        sum(actual_units_sold_in_scope_2024) as actual_units_sold_in_scope_2024,

        sum(actual_sales_in_scope_l52) as actual_sales_in_scope_l52,
        sum(actual_units_sold_in_scope_l52) as actual_units_sold_in_scope_l52,
        sum(dnp_in_scope_l52) as dnp_in_scope_l52,

        -- DNP
        avg(dnp_ratio) as avg_dnp_ratio,
        median(dnp_ratio) as median_dnp_ratio,

        
        -- Unit sold
        sum(optimal_units_sold) as optimal_units_sold,
        sum(current_units_sold) as current_units_sold,
        sum(upcoming_units_sold) as upcoming_units_sold,
        sum(adjustment_max_fiscal_week) as adjustment_max_fiscal_week,
    
        sum(facing_total) as facing_total,
        sum(facings_in_scope) as facings_in_scope,
        sum(facings_out_of_scope) as facings_out_of_scope,

        sum(linear_space_total) as linear_space_total,
        sum(linear_space_in_scope) as linear_space_in_scope,
        sum(linear_space_out_of_scope) as linear_space_out_of_scope,
        
        
        sum(sales_total) as sales_total,
        sum(sales_out_of_scope) as sales_out_of_scope,

        sum(units_sold_total) as units_sold_total,
        sum(units_sold_out_of_scope) as units_sold_out_of_scope,
        
        sum(pods_total) as pods_total,
        sum(pod_out_of_scope) as pod_out_of_scope,

        sum(dnp_total) as dnp_total,
        sum(dnp_out_of_scope) as dnp_out_of_scope,

        sum(actual_sales_total_2024) as actual_sales_total_2024,
        sum(actual_sales_out_of_scope_2024) as actual_sales_out_of_scope_2024,
        
        
        -- opt + out of scope
         sum(adj_opti_plus_out_of_scope_sales) as adj_opti_plus_out_of_scope_sales,
         sum(adj_opti_plus_out_of_scope_dnp) as adj_opti_plus_out_of_scope_dnp,
         sum(adj_opti_plus_out_of_scope_units_sold) as adj_opti_plus_out_of_scope_units_sold,
         sum(adj_opti_plus_out_of_scope_pods) as adj_opti_plus_out_of_scope_pods,
         sum(adj_opti_plus_out_of_scope_facings) as adj_opti_plus_out_of_scope_facings,
         sum(adj_opti_plus_out_of_scope_linear_space) as adj_opti_plus_out_of_scope_linear_space,

         -- current + out of scope
         sum(current_plus_out_of_scope_facings) as current_plus_out_of_scope_facings
FROM item_pano_df
GROUP BY ALL
)

SELECT *
FROM item_pano_df
-- FROM vendor_level_df
"""

In [ ]:
item_level_df = conn.cursor().execute(SKU_level_query).fetch_pandas_all()
item_level_df.columns = map(str.lower, item_level_df.columns)

In [ ]:
item_level_df[["n_current_linear_space_used_sku","optimal_linear_space"]].sum()

### 5.2 fixture level

In [ ]:
fixture_level_query = f"""
WITH OPT_DF as (
SELECT *
FROM DL_FSCA_SLFSRV.TWA07.{context.scaling_sales.output_tables.final_opti_output_scaled}
WHERE TRUE
AND RUN_ID = '{context.meta.run_id}'
)
,
ITEM_DESCRIPTION AS (
    SELECT DISTINCT 
        SKU_NBR AS       item_no_nbr,
        CVS_ITEM_DSC as  item_no_dsc,
        CAT_DSC AS CATEGORY,
        seg_dsc as segment,
        VENDOR_NBR,
        VENDOR_NAME,
        CM_BRAND_DSC AS BRAND_NAME,
        SUBCAT_DSC AS SUB_CATEGORY,
        UNIT_WTH/12 AS ITEM_WIDTH_FEET
    FROM CORE_FS.CURATED_PRODUCT.SKU
), 
-- CTE 1: Get the full scope
FULL_SCOPE as (
SELECT DISTINCT SKU_NBR, PLANOGRAM_DSC
FROM DL_FSCA_SLFSRV.TWA07.{ACTUAL_SALES_TABLE_CATEGORY}
UNION
SELECT DISTINCT item_no_nbr AS SKU_NBR, fixture_desc AS PLANOGRAM_DSC
FROM OPT_DF
)
,
-- CTE 2: Aggregate metrics at item level
ITEM_AGGREGATE_METRICS AS (
    SELECT 
        'baseline' as   run_id, 
        run_id as       raw_run_id, 
        fixture_desc AS PLANOGRAM_DSC, 
        item_no_nbr as  item_no_nbr,
        
        -- POD (Plan-O-Gram Distribution) calculations
        -- Count stores where item has facings in each state
        SUM(CASE WHEN n_optimal_facings_sku > 0 THEN n_stores ELSE 0 END) AS OPTIMAL_POD,
        SUM(n_optimal_facings_sku * n_stores) AS OPTIMAL_FACING_COUNT,

        SUM(CASE WHEN n_current_facings_sku > 0 THEN n_stores ELSE 0 END) AS CURRENT_POD,
        SUM(n_current_facings_sku * n_stores) AS CURRENT_FACING_COUNT,

        SUM(CASE WHEN n_upcoming_facings_sku > 0 THEN n_stores ELSE 0 END) AS UPCOMING_POD,
        SUM(n_upcoming_facings_sku * n_stores) AS UPCOMING_FACING_COUNT,
        
        -- Sales projections (normalized by store count)
        SUM(n_optimal_scaled_adjusted_projected_sales  * n_stores) AS normalized_optimal_adjusted_projected_sales,
        SUM(n_current_scaled_adjusted_projected_sales  * n_stores) AS normalized_current_adjusted_projected_sales,
        SUM(n_upcoming_scaled_adjusted_projected_sales * n_stores) AS normalized_upcoming_adjusted_projected_sales,
        
        -- DNP projections
        SUM(n_optimal_scaled_adjusted_projected_dnp  * n_stores) AS normalized_optimal_adjusted_projected_dnp,
        SUM(n_current_scaled_adjusted_projected_dnp  * n_stores) AS normalized_current_adjusted_projected_dnp,
        SUM(n_upcoming_scaled_adjusted_projected_dnp * n_stores) AS normalized_upcoming_adjusted_projected_dnp,

        -- Unit sold
        SUM(n_optimal_scaled_adjusted_projected_units_sold  * n_stores) as optimal_units_sold,
        SUM(n_current_scaled_adjusted_projected_units_sold  * n_stores) as current_units_sold,
        SUM(n_upcoming_scaled_adjusted_projected_units_sold * n_stores) as upcoming_units_sold,
        
        --
        SUM(n_optimal_linear_space_used_sku * n_stores) as optimal_linear_space,
        sum(n_current_linear_space_used_sku * n_stores) as n_current_linear_space_used_sku,
        min({context.scaling_sales.time_period.week_nbr_end}) as adjustment_max_fiscal_week,
        
        MIN(dnp_ratio) AS dnp_ratio  -- Take minimum DNP ratio for the item
    FROM OPT_DF
    GROUP BY ALL
)

, 
GRANULARITY as (
SELECT DISTINCT
        'baseline' as run_id, 
        run_id as raw_run_id,
        'baseline' AS SCENEARIO_OPTIMIZATION_VENDOR,
        '' AS SPACE_CHANGE,
        '-' AS PCT_SPACE_CHANGE
        FROM OPT_DF
)
,
-- CTE 3: Base item information from optimization runs
BASE_ITEM_TABLE AS (
    SELECT DISTINCT 
        'baseline' as run_id, 
        run_id as raw_run_id,
        'baseline' AS SCENEARIO_OPTIMIZATION_VENDOR,
        '' AS SPACE_CHANGE,
        '-' AS PCT_SPACE_CHANGE,
        fixture_desc AS PLANOGRAM_DSC,
        item_no_nbr AS item_no_nbr,
        item_no_desc AS ITEM_NO_DSC,
        seg_dsc AS SEGMENT,
        -- Extract choice map from need state ID
        REGEXP_SUBSTR(need_state_unique_id, '^(.*)_', 1, 1, 'e') AS CHOICE_MAP
    FROM OPT_DF
)
, 
-- CTE 4: Get unit prices for unit sales calculations
UNIT_PRICE_TBL AS (
    SELECT DISTINCT
        UPC AS ITEM_NO_NBR,
        SKU_MSRP AS UNIT_PRICE
    FROM {context.data_stores.raw_data.table_name.new_skus_table}
    WHERE UPC not in (SELECT SKU_NBR FROM DL_FSCA_SLFSRV.TWA07.STORE_SKU_UNIT_PRICE)
    
    UNION
    
    SELECT 
        SKU_NBR AS item_no_nbr,
        MEDIAN(L52W_PRICE) AS UNIT_PRICE
    FROM DL_FSCA_SLFSRV.TWA07.STORE_SKU_UNIT_PRICE
    GROUP BY SKU_NBR
)
, 
-- CTE 5: Get actual sales data from comprehensive sales table
RELEVANT_ACTUALS AS (
    SELECT 
        f.SKU_NBR AS item_no_nbr,
        sku.category_dsc AS CATEGORY,
        f.PLANOGRAM_DSC,
        coalesce(s.TOTAL_SALES_FULL_SCOPE_L52,0)       AS TOTAL_SALES_FULL_SCOPE_L52,
        coalesce(s.TOTAL_UNITS_SOLD_FULL_SCOPE_L52,0)  AS TOTAL_UNITS_SOLD_FULL_SCOPE_L52,
        coalesce(s.TOTAL_SALES_FULL_SCOPE_2024,0)      AS TOTAL_SALES_FULL_SCOPE_2024,
        coalesce(s.TOTAL_UNITS_SOLD_FULL_SCOPE_2024,0) AS TOTAL_UNITS_SOLD_FULL_SCOPE_2024,
        coalesce(s.ACTUAL_SALES_IN_SCOPE_2024,0)       AS ACTUAL_SALES_IN_SCOPE_2024,
        coalesce(s.ACTUAL_UNITS_SOLD_IN_SCOPE_2024,0)  AS ACTUAL_UNITS_SOLD_IN_SCOPE_2024,
        coalesce(s.ACTUAL_SALES_IN_SCOPE_L52,0)        AS ACTUAL_SALES_IN_SCOPE_L52,
        coalesce(s.ACTUAL_UNITS_SOLD_IN_SCOPE_L52,0)   AS ACTUAL_UNITS_SOLD_IN_SCOPE_L52,
        coalesce(s.ITEM_LATEST_DNP_RATIO, NULL)        AS ITEM_LATEST_DNP_RATIO
    FROM FULL_SCOPE f
    LEFT JOIN DL_FSCA_SLFSRV.TWA07.{ACTUAL_SALES_TABLE_CATEGORY} s 
    on f.SKU_NBR = s.sku_nbr and f.PLANOGRAM_DSC = s.PLANOGRAM_DSC
    LEFT JOIN CORE_FSSC.CURATED_PRODUCT.SKU sku on f.SKU_NBR = sku.sku_nbr
)
,

-- Linear space
LINEAR_SPACE_ACTUAL as (
SELECT  f.SKU_NBR AS item_no_nbr,
        f.PLANOGRAM_DSC,
        coalesce(FACING_TOTAL,0) AS FACING_TOTAL,
        coalesce(FACINGS_IN_SCOPE,0) AS FACINGS_IN_SCOPE,
        coalesce(FACINGS_OUT_OF_SCOPE,0) AS FACINGS_OUT_OF_SCOPE,
        coalesce(LINEAR_SPACE_TOTAL,0) AS LINEAR_SPACE_TOTAL,
        coalesce(LINEAR_SPACE_IN_SCOPE,0) AS LINEAR_SPACE_IN_SCOPE,
        coalesce(LINEAR_SPACE_OUT_OF_SCOPE,0) AS LINEAR_SPACE_OUT_OF_SCOPE,
        coalesce(PODS_TOTAL,0) AS PODS_TOTAL,
        coalesce(POD_IN_SCOPE,0) AS POD_IN_SCOPE,
        coalesce(POD_OUT_OF_SCOPE,0) AS POD_OUT_OF_SCOPE
FROM FULL_SCOPE f
LEFT JOIN DL_FSCA_SLFSRV.TWA07.{ACTUAL_LINEAR_SPACE_TABLE} s
on f.SKU_NBR = s.item_no_nbr and f.PLANOGRAM_DSC = s.PLANOGRAM_NAME
)
,
-- new items
new_item_df as
(
SELECT DISTINCT
    n.*,
    v.vendor_nbr, 
    pv.pay_vendor_nm
FROM {context.data_stores.raw_data.table_name.new_skus_table} n
LEFT JOIN CORE_FSSC.CURATED_PRODUCT.VENDOR v on n.vendor_number = v.vendor_nbr
LEFT JOIN CORE_FSSC.CURATED_PRODUCT.PAY_VENDOR pv on v.pay_vendor_nbr = pv.pay_vendor_nbr
)

,
item_pano_df as 
(SELECT  
        -- GRANULARITY
        G.run_id, 
        G.SCENEARIO_OPTIMIZATION_VENDOR as sceneario_optimization_vendor,
        G.SPACE_CHANGE as space_change,
        G.PCT_SPACE_CHANGE as pct_space_change,
        RA.PLANOGRAM_DSC as planogram_dsc,
        -- Actauls
        RA.item_no_nbr,
        coalesce(ID.item_no_dsc, ND.product_name) as  item_no_dsc,
        coalesce(RA.CATEGORY, ND.category) as category,
        coalesce(ID.SUB_CATEGORY, ND.subcategory) as subcategory,
        coalesce(ID.segment, ND.segment) as segment,
        B.CHOICE_MAP as choice_map,
        coalesce(pv.pay_vendor_nm, ND.pay_vendor_nm) as pay_vendor_name,
        coalesce(ID.BRAND_NAME, ND.brand_name) as brand_name,
        ID.ITEM_WIDTH_FEET as item_width_feet,
        
        -- ITEM CATEGORY, ATTRIBUTES
        IAM.OPTIMAL_FACING_COUNT  AS projected_optimal_facing_count,
        IAM.OPTIMAL_POD           AS projected_optimal_pod,
        IAM.CURRENT_FACING_COUNT  AS projected_current_facing_count,
        IAM.CURRENT_POD           AS projected_current_pod,
        IAM.UPCOMING_FACING_COUNT AS projected_upcoming_facing_count,
        IAM.UPCOMING_POD          AS projected_upcoming_pod,
         -- Sales projections
        IAM.normalized_optimal_adjusted_projected_sales as normalized_optimal_projected_sales,
        IAM.normalized_optimal_adjusted_projected_sales,
        IAM.normalized_current_adjusted_projected_sales as normalized_current_projected_sales,
        IAM.normalized_current_adjusted_projected_sales,
        IAM.normalized_upcoming_adjusted_projected_sales as normalized_upcoming_projected_sales,
        IAM.normalized_upcoming_adjusted_projected_sales,
         -- DNP projections
        IAM.normalized_optimal_adjusted_projected_dnp as normalized_optimal_projected_dnp,
        IAM.normalized_optimal_adjusted_projected_dnp,
        IAM.normalized_current_adjusted_projected_dnp as normalized_current_projected_dnp,
        IAM.normalized_current_adjusted_projected_dnp,
        IAM.normalized_upcoming_adjusted_projected_dnp as normalized_upcoming_projected_dnp,
        IAM.normalized_upcoming_adjusted_projected_dnp,
        

        -- DNP
        coalesce(RA.ITEM_LATEST_DNP_RATIO,IAM.dnp_ratio) as dnp_ratio,
        UPT.UNIT_PRICE as unit_price,

        -- Actuals - In scope
        
        LSA.POD_IN_SCOPE as actual_qty_stores_in_scope,
        LSA.POD_IN_SCOPE as actual_pods_in_scope,
        RA.ACTUAL_SALES_IN_SCOPE_2024 as actual_sales_in_scope_2024,
        RA.ACTUAL_SALES_IN_SCOPE_2024 * RA.ITEM_LATEST_DNP_RATIO as dnp_in_scope_2024,
        RA.ACTUAL_UNITS_SOLD_IN_SCOPE_2024 as actual_units_sold_in_scope_2024,

        RA.ACTUAL_SALES_IN_SCOPE_L52 as  actual_sales_in_scope_l52,
        RA.ACTUAL_UNITS_SOLD_IN_SCOPE_L52 as actual_units_sold_in_scope_l52,
        RA.ACTUAL_SALES_IN_SCOPE_L52 * RA.ITEM_LATEST_DNP_RATIO as dnp_in_scope_l52,

        IAM.optimal_units_sold,
        IAM.current_units_sold,
        IAM.upcoming_units_sold,
        IAM.adjustment_max_fiscal_week,
    
        LSA.FACING_TOTAL as facing_total,
        LSA.FACINGS_IN_SCOPE as facings_in_scope,
        LSA.FACINGS_OUT_OF_SCOPE as facings_out_of_scope,

        LSA.LINEAR_SPACE_TOTAL        as linear_space_total,
        LSA.LINEAR_SPACE_IN_SCOPE     as linear_space_in_scope,
        LSA.LINEAR_SPACE_OUT_OF_SCOPE as linear_space_out_of_scope,
        
        
        RA.TOTAL_SALES_FULL_SCOPE_L52 as sales_total,
        RA.TOTAL_SALES_FULL_SCOPE_L52 - RA.ACTUAL_SALES_IN_SCOPE_L52 as sales_out_of_scope,

        RA.TOTAL_UNITS_SOLD_FULL_SCOPE_L52 as units_sold_total,
        RA.TOTAL_UNITS_SOLD_FULL_SCOPE_L52 - RA.ACTUAL_UNITS_SOLD_IN_SCOPE_L52 as units_sold_out_of_scope,
        
        LSA.PODS_TOTAL as pods_total,
        LSA.POD_OUT_OF_SCOPE as pod_out_of_scope,

        RA.TOTAL_SALES_FULL_SCOPE_L52 * coalesce(RA.ITEM_LATEST_DNP_RATIO,IAM.dnp_ratio) as dnp_total,
        (RA.TOTAL_SALES_FULL_SCOPE_L52 - RA.ACTUAL_SALES_IN_SCOPE_L52) * coalesce(RA.ITEM_LATEST_DNP_RATIO,IAM.dnp_ratio) as dnp_out_of_scope,

        RA.TOTAL_SALES_FULL_SCOPE_2024 as actual_sales_total_2024,
        RA.TOTAL_SALES_FULL_SCOPE_2024 - RA.ACTUAL_SALES_IN_SCOPE_2024 as actual_sales_out_of_scope_2024,
        
        
        -- opt + out of scope
         coalesce(IAM.normalized_optimal_adjusted_projected_sales,0) + (RA.TOTAL_SALES_FULL_SCOPE_L52 - RA.ACTUAL_SALES_IN_SCOPE_L52) as adj_opti_plus_out_of_scope_sales,
         coalesce(IAM.normalized_optimal_adjusted_projected_dnp,0) +   (RA.TOTAL_SALES_FULL_SCOPE_L52 - RA.ACTUAL_SALES_IN_SCOPE_L52) * coalesce(RA.ITEM_LATEST_DNP_RATIO,IAM.dnp_ratio)  as adj_opti_plus_out_of_scope_dnp,
         coalesce(IAM.optimal_units_sold,0) + (RA.TOTAL_UNITS_SOLD_FULL_SCOPE_L52 - RA.ACTUAL_UNITS_SOLD_IN_SCOPE_L52) as adj_opti_plus_out_of_scope_units_sold,
         coalesce(IAM.OPTIMAL_POD,0) + LSA.POD_OUT_OF_SCOPE as adj_opti_plus_out_of_scope_pods,
         coalesce(IAM.OPTIMAL_FACING_COUNT,0) + LSA.FACINGS_OUT_OF_SCOPE as adj_opti_plus_out_of_scope_facings,
         coalesce(IAM.optimal_linear_space,0) + LSA.LINEAR_SPACE_OUT_OF_SCOPE  as adj_opti_plus_out_of_scope_linear_space,
         coalesce(IAM.optimal_linear_space,0) as optimal_linear_space,
         coalesce(IAM.n_current_linear_space_used_sku,0) as n_current_linear_space_used_sku,

        -- current + out of scope
         coalesce(IAM.CURRENT_FACING_COUNT,0) + coalesce(LSA.FACINGS_OUT_OF_SCOPE,0) as current_plus_out_of_scope_facings
FROM RELEVANT_ACTUALS RA
CROSS JOIN GRANULARITY G
LEFT JOIN ITEM_DESCRIPTION ID ON RA.item_no_nbr = ID.item_no_nbr
LEFT JOIN CORE_FSSC.CURATED_PRODUCT.VENDOR v on ID.vendor_nbr = v.vendor_nbr
LEFT JOIN CORE_FSSC.CURATED_PRODUCT.PAY_VENDOR pv on v.pay_vendor_nbr = pv.pay_vendor_nbr
LEFT JOIN new_item_df ND ON RA.item_no_nbr = ND.upc
LEFT JOIN UNIT_PRICE_TBL UPT on RA.item_no_nbr = UPT.item_no_nbr
LEFT JOIN LINEAR_SPACE_ACTUAL LSA 
ON RA.item_no_nbr = LSA.item_no_nbr and RA.PLANOGRAM_DSC = LSA.PLANOGRAM_DSC
LEFT JOIN ITEM_AGGREGATE_METRICS IAM
on RA.item_no_nbr = IAM.item_no_nbr and RA.PLANOGRAM_DSC = IAM.PLANOGRAM_DSC and G.run_id = IAM.run_id
LEFT JOIN BASE_ITEM_TABLE B
on RA.item_no_nbr = B.item_no_nbr and RA.PLANOGRAM_DSC = B.PLANOGRAM_DSC and G.run_id = B.run_id)

,
vendor_level_df as (
SELECT  
        run_id, 
        planogram_dsc,
        SPLIT_PART(run_id, '_', 2) AS SCENEARIO_OPTIMIZATION_VENDOR,
        SPLIT_PART(run_id, '_', 4) AS SPACE_CHANGE,
        CONCAT(SPLIT_PART(run_id, '_', 5), '-', SPLIT_PART(run_id, '_', 6)) AS PCT_SPACE_CHANGE,
            -- Sales projections
        sum(normalized_optimal_projected_sales) as normalized_optimal_projected_sales,
        sum(normalized_optimal_adjusted_projected_sales) as normalized_optimal_adjusted_projected_sales,
        sum(normalized_current_projected_sales) as normalized_current_projected_sales,
        sum(normalized_current_adjusted_projected_sales) as normalized_current_adjusted_projected_sales,
        sum(normalized_upcoming_projected_sales) as normalized_upcoming_projected_sales,
        sum(normalized_upcoming_adjusted_projected_sales) as normalized_upcoming_adjusted_projected_sales,
         -- DNP projections
        sum(normalized_optimal_projected_dnp) as normalized_optimal_projected_dnp,
        sum(normalized_optimal_adjusted_projected_dnp) as normalized_optimal_adjusted_projected_dnp,
        sum(normalized_current_projected_dnp) as normalized_current_projected_dnp,
        sum(normalized_current_adjusted_projected_dnp) as normalized_current_adjusted_projected_dnp,
        sum(normalized_upcoming_projected_dnp) as normalized_upcoming_projected_dnp,
        sum(normalized_upcoming_adjusted_projected_dnp) as normalized_upcoming_adjusted_projected_dnp,
        

        -- Actuals - In scope
        
        sum(actual_qty_stores_in_scope) as actual_qty_stores_in_scope,
        sum(actual_pods_in_scope) as actual_pods_in_scope,
        sum(actual_sales_in_scope_2024) as actual_sales_in_scope_2024,
        sum(dnp_in_scope_2024) as dnp_in_scope_2024,
        sum(actual_units_sold_in_scope_2024) as actual_units_sold_in_scope_2024,

        sum(actual_sales_in_scope_l52) as actual_sales_in_scope_l52,
        sum(actual_units_sold_in_scope_l52) as actual_units_sold_in_scope_l52,
        sum(dnp_in_scope_l52) as dnp_in_scope_l52,

        -- DNP
        avg(dnp_ratio) as avg_dnp_ratio,
        median(dnp_ratio) as median_dnp_ratio,

        
        -- Unit sold
        sum(optimal_units_sold) as optimal_units_sold,
        sum(current_units_sold) as current_units_sold,
        sum(upcoming_units_sold) as upcoming_units_sold,
        sum(adjustment_max_fiscal_week) as adjustment_max_fiscal_week,
    
        sum(facing_total) as facing_total,
        sum(facings_in_scope) as facings_in_scope,
        sum(facings_out_of_scope) as facings_out_of_scope,

        sum(linear_space_total) as linear_space_total,
        sum(linear_space_in_scope) as linear_space_in_scope,
        sum(linear_space_out_of_scope) as linear_space_out_of_scope,
        
        
        sum(sales_total) as sales_total,
        sum(sales_out_of_scope) as sales_out_of_scope,

        sum(units_sold_total) as units_sold_total,
        sum(units_sold_out_of_scope) as units_sold_out_of_scope,
        
        sum(pods_total) as pods_total,
        sum(pod_out_of_scope) as pod_out_of_scope,

        sum(dnp_total) as dnp_total,
        sum(dnp_out_of_scope) as dnp_out_of_scope,

        sum(actual_sales_total_2024) as actual_sales_total_2024,
        sum(actual_sales_out_of_scope_2024) as actual_sales_out_of_scope_2024,
        
        
        -- opt + out of scope
         sum(adj_opti_plus_out_of_scope_sales) as adj_opti_plus_out_of_scope_sales,
         sum(adj_opti_plus_out_of_scope_dnp) as adj_opti_plus_out_of_scope_dnp,
         sum(adj_opti_plus_out_of_scope_units_sold) as adj_opti_plus_out_of_scope_units_sold,
         sum(adj_opti_plus_out_of_scope_pods) as adj_opti_plus_out_of_scope_pods,
         sum(adj_opti_plus_out_of_scope_facings) as adj_opti_plus_out_of_scope_facings,
         sum(adj_opti_plus_out_of_scope_linear_space) as adj_opti_plus_out_of_scope_linear_space,

         -- current + out of scope
         sum(current_plus_out_of_scope_facings) as current_plus_out_of_scope_facings
FROM item_pano_df
GROUP BY ALL
)

SELECT *
FROM vendor_level_df
"""

In [ ]:
fixture_level_df = conn.cursor().execute(fixture_level_query).fetch_pandas_all()
fixture_level_df.columns = map(str.lower, fixture_level_df.columns)

### 5.3 Item-level Raw Opti

In [ ]:
item_raw_opti_query = f"""
with needstate_df as (
SELECT *
FROM DL_FSCA_SLFSRV.TWA07.{need_state_table_wide}
)

SELECT  item_no_nbr as "Item No Nbr",
        item_no_desc as "Item No Desc",
        vendor_name as "Vendor Name",
        brand_name as "Brand Name",
        cdt as "CDT",
        need_state_unique_id as "Need State Unique Id",
        attribute_1 as "Attribute 1",
        attribute_2 as "Attribute 2",
        attribute_3 as "Attribute 3",
        attribute_4 as "Attribute 4",
        max(total_store_count) over() as "Total Store Count",
        current_store_count as "Current Store Count",
        optimal_store_count as "Optimal Store Count",
        CASE WHEN current_store_count = 0 or optimal_store_count = 0 then NULL ELSE store_count_pct_change END as "Store Count Pct. Change",
        current_facings as "Current Facings",
        optimal_facings as "Optimal Facings",
        CASE WHEN current_facings = 0 or optimal_facings = 0 THEN NULL ELSE facings_pct_change END as "Facings Pct. Change",
        current_weighted_avg_facings as "Current Weighted Avg Facings",
        optimal_weighted_avg_facings as "Optimal Weighted Avg Facings",
        CASE WHEN current_weighted_avg_facings = 0 THEN NULL ELSE weighted_avg_facings_pct_change END as "Weighted Avg Facings Pct. Change",
        new_item as "New Item",
        disc_item as "Disc Item"
FROM 
    (
    SELECT  a.item_no_nbr,
            a.item_no_desc,
            a.vendor_name,
            a.brand_name,
            c.cdt,
            a.need_state_unique_id,
            c.attribute_1,
            c.attribute_2,
            c.attribute_3,
            c.attribute_4,
            sum(n_stores) as total_store_count,
            sum(case when n_current_facings_sku > 0 THEN n_stores ELSE 0 END) as current_store_count,
            sum(case when n_optimal_facings_sku > 0 THEN n_stores ELSE 0 END) as optimal_store_count,
            div0(sum(case when n_optimal_facings_sku > 0 THEN n_stores ELSE 0 END),sum(case when n_current_facings_sku > 0 THEN n_stores ELSE 0 END)) - 1 as store_count_pct_change,
            sum(n_current_facings_sku * n_stores) current_facings,
            sum(n_optimal_facings_sku * n_stores) optimal_facings,
            div0(sum(n_optimal_facings_sku * n_stores), sum(n_current_facings_sku * n_stores)) - 1 as facings_pct_change,
            sum(n_current_facings_sku * n_stores)/sum(n_stores) as current_weighted_avg_facings,
            sum(n_optimal_facings_sku * n_stores)/sum(n_stores) as optimal_weighted_avg_facings,
            div0(sum(n_optimal_facings_sku * n_stores)/sum(n_stores), sum(n_current_facings_sku * n_stores)/sum(n_stores)) - 1 as weighted_avg_facings_pct_change,
            max(a.new_item) as new_item,
            max(a.discontinue_item) as disc_item
    FROM DL_FSCA_SLFSRV.TWA07.FNL_MICRO_OPTIMIZATION_RUNS_OB a
    LEFT JOIN CORE_FSSC.CURATED_PRODUCT.SKU b on a.item_no_nbr = b.sku_nbr
    LEFT JOIN needstate_df c on a.item_no_nbr = c.product_id
    WHERE TRUE
    AND run_id = '{context.meta.run_id}'
    GROUP BY ALL
    ORDER BY 1
    )
"""

item_raw_opti_df = conn.cursor().execute(item_raw_opti_query).fetch_pandas_all()

In [ ]:
os.makedirs('./scripts/scaling_output', exist_ok=True)

with pd.ExcelWriter(f'./scripts/scaling_output/scaled_output_{CAT_NAME}_{date}.xlsx') as writer:
    # Write the first DataFrame to 'Sheet1' without the index
    item_level_df.to_excel(writer, sheet_name='item_level', index=False)

    # Write the second DataFrame to 'Sheet2' without the index
    fixture_level_df.to_excel(writer, sheet_name='fixture_level', index=False)

    # Write the second DataFrame to 'Sheet2' without the index
    item_raw_opti_df.to_excel(writer, sheet_name='item-raw opti', index=False)

### 5.4 Baseline Opti

In [ ]:
q = f"""
SELECT *
FROM DL_FSCA_SLFSRV.TWA07.FNL_MICRO_OPTIMIZATION_RUNS_OB
WHERE TRUE
AND run_id = '{context.meta.run_id}'
"""
raw_opti_baseline = conn.cursor().execute(q).fetch_pandas_all()


In [ ]:
raw_opti_baseline.to_csv(f"./scripts/scaling_output/final_output_{CAT_NAME}_{context.meta.run_id}.csv")